<a href="https://colab.research.google.com/github/seedhecode75/IITG-BSBE-Archive/blob/main/medguide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 datasets


In [2]:
!pip install --upgrade bitsandbytes


  Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl (76.1 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.39.0
    Uninstalling bitsandbytes-0.39.0:
      Successfully uninstalled bitsandbytes-0.39.0


In [ ]:
import os
os.kill(os.getpid(), 9)

In [5]:
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


In [1]:
!pip install --upgrade pip
!pip install bitsandbytes==0.39.0
!pip install transformers==4.31.0
!pip install accelerate==0.21.0
!pip install peft==0.4.0
!pip install trl==0.4.7


  Using cached bitsandbytes-0.39.0-py3-none-any.whl.metadata (9.8 kB)
Using cached bitsandbytes-0.39.0-py3-none-any.whl (92.2 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.45.5
    Uninstalling bitsandbytes-0.45.5:
      Successfully uninstalled bitsandbytes-0.45.5


In [2]:
from datasets import load_dataset

data = load_dataset(
    "Mohammed-Altaf/medical-instruction-120k",
    data_files="medicare_110k_train.json",
    split="train"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [6]:
ds = load_dataset("heliosbrahma/mental_health_chatbot_dataset")


NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [8]:
import pandas as pd

# Mount Drive if needed
from google.colab import drive
drive.mount('/content/drive')

# Load your CSV
df = pd.read_csv('/content/drive/MyDrive/medicare_110k_test.csv')
dataset = Dataset.from_pandas(df)



Mounted at /content/drive


In [10]:
df


,Conversation
0,The conversation between human and AI assistan...
1,The conversation between human and AI assistan...
2,The conversation between human and AI assistan...
3,The conversation between human and AI assistan...
4,The conversation between human and AI assistan...
...,...
5604,The conversation between human and AI assistan...
5605,The conversation between human and AI assistan...
5606,The conversation between human and AI assistan...
5607,The conversation between human and AI assistan...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5609 entries, 0 to 5608
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Conversation  5609 non-null   object
dtypes: object(1)
memory usage: 43.9+ KB


In [12]:
import re

def extract_prompt_response(text):
    # Remove unwanted line breaks and carriage returns
    text = text.replace('_x000D_', ' ').replace('\n', ' ').strip()
    # Extract prompt and response using regex
    match = re.search(r'\[\|Human\|\](.*?)\[\|AI\|\](.*)', text)
    if match:
        prompt = match.group(1).strip()
        response = match.group(2).strip()
        return prompt, response
    else:
        return None, None

# Apply extraction to each row
df[['prompt', 'response']] = df.iloc[:, 0].apply(lambda x: pd.Series(extract_prompt_response(str(x))))
# Drop rows where extraction failed
df = df.dropna(subset=['prompt', 'response'])
df.head()


,Conversation,prompt,response
0,The conversation between human and AI assistan...,"I wake in the night, usually about 2-3 hours a...",Dear patient Here are the possibilities of wha...
1,The conversation between human and AI assistan...,"Honorable Sir,I am Ripon Dev from Bangladesh.M...","Hi, Thanks for writing in. Please add detailed..."
2,The conversation between human and AI assistan...,Ive had a cold which started on Christmas eve ...,"Hi, Welcome to Chat Doctor! Yes, from what you..."
3,The conversation between human and AI assistan...,I had cervical laminectomy surgery for spinal ...,Thanks for writing to us. You have complex sto...
4,The conversation between human and AI assistan...,"Hello, At the end of lacrosse practice about a...","Dear List, I believe you may have suffered a m..."


In [13]:
cleaned_path = '/content/drive/MyDrive/cleaned_medical_dialogues.csv'
df[['prompt', 'response']].to_csv(cleaned_path, index=False)


In [15]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files=cleaned_path, split='train')
print(dataset[0])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/download/streaming_download_manager.py:765: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [16]:
# Mount Drive if needed
from google.colab import drive
drive.mount('/content/drive')

# Load your CSV
df = pd.read_csv(cleaned_path)
dataset = Dataset.from_pandas(df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
print(dataset)

Dataset({
    features: ['prompt', 'response'],
    num_rows: 5609
})


In [22]:
model_name = "tiiuae/falcon-7b"  # Or another compatible model
output_dir = "./qlora-finetuned"
new_model = "falcon-7b-qlora-finetuned"

# QLoRA parameters
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# bitsandbytes parameters
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

# Training parameters
num_train_epochs = 1
per_device_train_batch_size = 2
gradient_accumulation_steps = 4
learning_rate = 2e-4
save_steps = 50
logging_steps = 10
max_steps = 150


In [23]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
model.config.use_cache = False


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [24]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    save_steps=save_steps,
    logging_steps=logging_steps,
    max_steps=max_steps,
    fp16=True,  # Set bf16=True if using A100
    report_to="none",
)


In [29]:
def make_text(example):
    return {
        "text": f"[|Human|] {example['prompt']}\n[|AI|] {example['response']}"
    }

dataset = dataset.map(make_text)



Map:   0%|          | 0/5609 [00:00<?, ? examples/s]

In [30]:
print(dataset[:5])

{'prompt': ['I wake in the night, usually about 2-3 hours after going to sleep, with both feet and legs to mid calf feeling like they are on fire. slight red discolorization, minor swelling. This is very painful but after getting up, I can walk it off in about 30 minutes.', 'Honorable Sir,I am Ripon Dev from Bangladesh.My brother Prodip is a patient of kidney.Now we want to take treatment from you.For your kind information I am sending all medical documents of my brother.So requesting you to inform me what I have to do to take treatment from you.And also inform me about probable cost.Ripon DevDhakaBangladesh', 'Ive had a cold which started on Christmas eve but appeared to be getting better over the following week. However I now have what I think may be sinusitis - pain in the head, yellow mucus from the nose and stuffiness-squeaking from the sinuses. Will this go away on its own or should I see my GP?', 'I had cervical laminectomy surgery for spinal stenosis in Aug 2010. Since then I h

In [35]:
lora_alpha = 32 # scaling factor for the weight matrices
lora_dropout = 0.05 # dropout probability of the LoRA layers
lora_rank = 32 # dimension of the low-rank matrices

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM",
    target_modules=[         # Setting names of modules in falcon-7b model that we want to apply LoRA to
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)




In [37]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)




Map:   0%|          | 0/5609 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:427: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [38]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

In [39]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: s-boyidi (s-boyidi-indian-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [1]:

trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
from transformers import pipeline

# Load base model and merge LoRA adapters
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=200)

# Example inference
prompt = "Your prompt here"
output = pipe(prompt)
print(output[0]['generated_text'])
